# web interaction ds4 mgrs lab

In [12]:
import os, sys
import pprint
import subprocess
import sys
import string
import time
#from colorama import init, Fore, Back, Style
import colorama as cl
import pandas as pd
import splits_aggregator as sa
import extract_rules as er
import pq 
from interact import main

In [13]:
__author__ = 'John Mark Agosta john-mark.agosta@microsoft.com'

### config constants 
VERBOSE = False
RULES_PER_SAMPLE = 1
RULE_PAIRS =100
ROOT_DIR = Path('C:/Users/joagosta/OneDrive - Microsoft/data/20news')
INTERACTIVE = True

In [14]:
def main(root_dir, run_interactive):

    print('PID:', os.getpid())
    # Test split pattern extraction
    # input_pattern= glob_pattern + '*.csv'
    rules_dir = root_dir / "rules"
    cs = sa.BinaryComparisons(root_dir)
    # pair_df = cs.random_pairs(INTERACTIVE_RULE_PAIRS) # 
    # Save pairs to shared directory 
    # cs.embed_in_excel(pair_df)
    # Seed the model with a few auto-generated splits
    seed_df = cs.random_pairs(RULE_PAIRS)
    the_rules = cs.simulate_splits(seed_df)  # Creates simulated rules. 
    # While running interactively load new excel splits and add them to the rules
    # Load the test set
    test_df = pq.reload_parquet(root_dir / 'test_clean') 
    cl.init()
    summary_df = pd.DataFrame(columns = ['Train_acc','Train_pr', 'Train_rcl', 'Test_acc', 'Test_pr', 'Test_rcl'])
    user_rule_cnt = -1
    while INTERACTIVE:
        # Check if the rules directory has any new files, then continue. 
        # new_rule_cnt = len(list(rules_dir.glob( SS_SUFFIX)))
        new_rule_cnt = 0
        if new_rule_cnt > user_rule_cnt:
            #users = sa.CollectSplits(rules_dir, SS_SUFFIX)
            user_rules = er.extract_rules()
            user_rules.extend(the_rules)  # combine lists
            if VERBOSE: pprint.pprint(the_rules)
            # Evaluate the ruleset and display. 
            learner = sa.SplitClassifier(the_rules)
            # learner.order_by_hits(cs.full_df)  # Don't need to tune the rules 
            summary_train = learner.compute_confusion(cs.full_df)
            # Do the same for the test data
            summary_test = learner.compute_confusion(test_df)
            summary_train.extend(summary_test)
            # Collect summary
            next_episode = pd.DataFrame([summary_train])
            next_episode.columns = ['Train_acc','Train_pr', 'Train_rcl', 'Test_acc', 'Test_pr', 'Test_rcl']
            summary_df = summary_df.append(next_episode, ignore_index=True)
            print(cl.Fore.YELLOW + cl.Style.BRIGHT + 'Summary') 
            print(summary_df)
            print(cl.Fore.RESET)
            # user_rule_cnt = new_rule_cnt
        else:
            time.sleep(10.0)
    summary_df.to_csv(Path(root_dir)/'summary.csv', header=True, index=False)
    return 0

In [15]:
main(ROOT_DIR, INTERACTIVE)


PID: 22468


Reloaded C:\Users\joagosta\OneDrive - Microsoft\data\20news\train_clean\full_df.parquet


200  selection rules.


Reloaded C:\Users\joagosta\OneDrive - Microsoft\data\20news\test_clean\full_df.parquet


Accuracy on matches = 548  /  2736  =  0.20029239766081872
                          0   1   2   3   4   5   6   7   8   9   10  11  12  \
comp.graphics             22  24  11  12   0   1  16   5   2   7  10   0   2   
sci.med                    8  45   2  14   1   0  23   3   3  16  15   2   4   
comp.windows.x             5  21  52  23   0   1   9   0   0   4  12   0   1   
misc.forsale              19   4   1  44   0   1   4   3   0   0   3   4   1   
rec.sport.hockey           5   4   0   8  22   0  17   0   1   1   6   2   0   
comp.os.ms-windows.misc    4  17  11   2   0   7  11   1   0   2   7   0   0   
rec.sport.baseball         6   7   1   8   3   0  35   0   0   4   6   0   0   
comp.sys.mac.hardware      7  14   9  13   0   0  23  25   0   1  12   6   0   
soc.religion.christian     7  13   3  15   0   2  16   3  21  25  21   3   2   
talk.politics.guns         5  10   3   6   0   0  11   1   3  21   8   2   7   
sci.electronics            5  29  10   7   0   2  19   1   0 

...

sci.crypt                  0   3  10   8   7   0  23  
                              prec    recall         F    sup
nms                                                          
comp.graphics             0.158273  0.051044  0.077193  431.0
sci.med                   0.165441  0.088583  0.115385  508.0
comp.windows.x            0.385185  0.120930  0.184071  430.0
misc.forsale              0.192140  0.126074  0.152249  349.0
rec.sport.hockey          0.814815  0.047930  0.090535  459.0
comp.os.ms-windows.misc   0.368421  0.015625  0.029979  448.0
rec.sport.baseball        0.104790  0.075922  0.088050  461.0
comp.sys.mac.hardware     0.423729  0.052743  0.093809  474.0
soc.religion.christian    0.420000  0.044211  0.080000  475.0
talk.politics.guns        0.125000  0.051471  0.072917  408.0
sci.electronics           0.123853  0.055901  0.077033  483.0
talk.politics.misc        0.130435  0.017391  0.030691  345.0
talk.politics.mideast     0.589744  0.108747  0.183633  423.0
rec.autos      

40   0.200292  0.275005   0.063889  0.123729  0.181492  0.038561

Accuracy on matches = 548  /  2736  =  0.20029239766081872
                          0   1   2   3   4   5   6   7   8   9   10  11  12  \
comp.graphics             22  24  11  12   0   1  16   5   2   7  10   0   2   
sci.med                    8  45   2  14   1   0  23   3   3  16  15   2   4   
comp.windows.x             5  21  52  23   0   1   9   0   0   4  12   0   1   
misc.forsale              19   4   1  44   0   1   4   3   0   0   3   4   1   
rec.sport.hockey           5   4   0   8  22   0  17   0   1   1   6   2   0   
comp.os.ms-windows.misc    4  17  11   2   0   7  11   1   0   2   7   0   0   
rec.sport.baseball         6   7   1   8   3   0  35   0   0   4   6   0   0   
comp.sys.mac.hardware      7  14   9  13   0   0  23  25   0   1  12   6   0   
soc.religion.christian     7  13   3  15   0   2  16   3  21  25  21   3   2   
talk.politics.guns         5  10   3   6   0   0  11   1   3  21   8   2   

AVGS                      0.181492  0.038561  0.055697  285.8
Summary
    Train_acc  Train_pr  Train_rcl  Test_acc   Test_pr  Test_rcl
0    0.200292  0.275005   0.063889  0.123729  0.181492  0.038561
1    0.200292  0.275005   0.063889  0.123729  0.181492  0.038561
2    0.200292  0.275005   0.063889  0.123729  0.181492  0.038561
3    0.200292  0.275005   0.063889  0.123729  0.181492  0.038561
4    0.200292  0.275005   0.063889  0.123729  0.181492  0.038561
5    0.200292  0.275005   0.063889  0.123729  0.181492  0.038561
6    0.200292  0.275005   0.063889  0.123729  0.181492  0.038561
7    0.200292  0.275005   0.063889  0.123729  0.181492  0.038561
8    0.200292  0.275005   0.063889  0.123729  0.181492  0.038561
9    0.200292  0.275005   0.063889  0.123729  0.181492  0.038561
10   0.200292  0.275005   0.063889  0.123729  0.181492  0.038561
11   0.200292  0.275005   0.063889  0.123729  0.181492  0.038561
12   0.200292  0.275005   0.063889  0.123729  0.181492  0.038561
13   0.200292  0.275

sci.crypt                  0   2   7   2   8   2   3  
                              prec    recall         F    sup
nms                                                          
comp.graphics             0.123596  0.039007  0.059299  282.0
sci.med                   0.126374  0.072327  0.092000  318.0
comp.windows.x            0.152542  0.033582  0.055046  268.0
misc.forsale              0.116129  0.086957  0.099448  207.0
rec.sport.hockey          0.500000  0.006309  0.012461  317.0
comp.os.ms-windows.misc   0.300000  0.019169  0.036036  313.0
rec.sport.baseball        0.046512  0.033113  0.038685  302.0
comp.sys.mac.hardware     0.361111  0.042623  0.076246  305.0
soc.religion.christian    0.320000  0.025157  0.046647  318.0
talk.politics.guns        0.094488  0.043011  0.059113  279.0
sci.electronics           0.046980  0.021672  0.029661  323.0
talk.politics.misc        0.045455  0.004329  0.007905  231.0
talk.politics.mideast     0.472222  0.059028  0.104938  288.0
rec.autos      

KeyboardInterrupt: 

In [ ]:
os.getpid()

In [ ]:
import psutil
pid = 22468  # replace this with the pid of your process
p = psutil.Process(pid)
p.suspend()
# ...to resume it:

#p.resume()